In [1]:
#Import required libraries
import numpy as np
import pandas as pd
import sklearn
import requests
from sklearn.preprocessing import OneHotEncoder

#This is the api key
api = "2c74fd25dbbaf91b73daf87263d0b35f"

In [2]:
#gen_list is the list of all possible genres in database

x = requests.get('https://api.themoviedb.org/3/genre/movie/list?api_key=2c74fd25dbbaf91b73daf87263d0b35f&language=en-US')
dfTemp = pd.json_normalize(x.json()['genres'])

gen_list = dfTemp["name"].values.tolist()

In [3]:

#This method will return a cleaned dataframe with the following specifications
#Additionally, this will only return movies whose original language is english
def get_clean_data(year = 2010, api = api, vote_count_min = 100):    
    df = pd.DataFrame([])
    IDs = pd.Series([],dtype = int)
    
    #retrieve the total number of pages that have movies that fit 
    x = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=%s&primary_release_year=%d&language=en-US&sort_by=vote_count.desc&include_adult=false&vote_count.gte=%d&include_video=false&page=%d&with_original_language=en'%(api,year,vote_count_min,1))
    totalPages = pd.json_normalize(x.json())["total_pages"].iloc[0]
    
    #print(totalPages)

    for pageNum in range(1,totalPages+1):
        #print(pageNum)
        #retrieve 20 movies
        x = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=%s&primary_release_year=%d&language=en-US&sort_by=vote_count.desc&include_adult=false&include_video=false&page=%d&vote_count.gte=%d&with_original_language=en'%(api,year,pageNum,vote_count_min))
        dfTemp = pd.json_normalize(x.json()['results'])
        #store all 20 movie ids into SerTemp Series
        SerTemp = pd.Series(dfTemp["id"])

        #loop through each of the 20 movie ids one by one
        for index, ids in SerTemp.items():
            #retrieve movie detals
            y = requests.get('https://api.themoviedb.org/3/movie/%d?api_key=%s&language=en-US'%(ids,api))
            dfTemp = pd.json_normalize(y.json())
            #keep only relevant columns
            dfTemp = dfTemp[["title","release_date","budget","revenue","vote_count","vote_average","runtime","id","genres"]]

            #ge
            genrelist = []
            for dic in dfTemp["genres"].iloc[0]:
                genrelist.append(dic.get('name'))

                dfTemp.at[0,"genres"]=genrelist


                allitems = dfTemp["genres"].iloc[0]
                for gen in gen_list:
                    colName = "genre_" + gen
                    if gen in allitems:
                        dfTemp[colName] = 1
                    else:
                        dfTemp[colName] = 0

            dfTemp.drop(["genres"],axis=1,inplace = True)

            df = pd.concat([df,dfTemp])
    
    return df



In [4]:

#this method will collect data from the api, clean it, and append it to a csv
#This is similar to the previous "get_clean_data" method
def write_clean_csv(csv_filename,append = True,year = 2010, api = api, vote_count_min = 100):    
    #df = pd.DataFrame([])
    IDs = pd.Series([],dtype = int)
    
    #create the file if it isn't already created
    #either erase the file or not depending on append variable
    if(append):
        file = open(csv_filename,'a',newline='')
        file.close()
    else:
        file = open(csv_filename,'w',newline='')
        file.close()
    
    #retrieve the total number of pages that have movies that fit 
    x = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=%s&primary_release_year=%d&language=en-US&sort_by=vote_count.desc&include_adult=false&vote_count.gte=%d&include_video=false&page=%d&with_original_language=en'%(api,year,vote_count_min,1))
    totalPages = pd.json_normalize(x.json())["total_pages"].iloc[0]
    
    #print(totalPages)

    for pageNum in range(1,totalPages+1):
        #print(pageNum)
        #retrieve 20 movies
        x = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=%s&primary_release_year=%d&language=en-US&sort_by=vote_count.desc&include_adult=false&include_video=false&page=%d&vote_count.gte=%d&with_original_language=en'%(api,year,pageNum,vote_count_min))
        dfTemp = pd.json_normalize(x.json()['results'])
        #store all 20 movie ids into SerTemp Series
        SerTemp = pd.Series(dfTemp["id"])
        
        #to reduce the number of times this writes, I create a new df to store 20 movie details
        df_movie_details = pd.DataFrame([])

        #loop through each of the 20 movie ids one by one
        for index, ids in SerTemp.items():
            #retrieve movie detals
            y = requests.get('https://api.themoviedb.org/3/movie/%d?api_key=%s&language=en-US'%(ids,api))
            dfTemp = pd.json_normalize(y.json())
            #keep only relevant columns
            dfTemp = dfTemp[["title","release_date","budget","revenue","vote_count","vote_average","runtime","id","genres"]]

            #ge
            genrelist = []
            for dic in dfTemp["genres"].iloc[0]:
                genrelist.append(dic.get('name'))

                dfTemp.at[0,"genres"]=genrelist


                allitems = dfTemp["genres"].iloc[0]
                for gen in gen_list:
                    colName = "genre_" + gen
                    if gen in allitems:
                        dfTemp[colName] = 1
                    else:
                        dfTemp[colName] = 0

            dfTemp.drop(["genres"],axis=1,inplace = True)
            
            #append each movie details line to df_movie_details
            df_movie_details = pd.concat([df_movie_details,dfTemp])
            
        
        #to ensure that the header appears once at the top, I set header to true if
        #it is the first iteration and a new file was created
        keepHeader = False
        if( not append and pageNum == 1):
            keepHeader = True
        
        #at the end of each 20 movies, take those details and write to csv
        df_movie_details.to_csv(csv_filename,mode = 'a',index = False, header = keepHeader)
    
    return

'hello'